In [1]:
####################################################################################
####################################################################################
####################################################################################
####################################################################################

In [2]:
'''
Initialize map 
'''

def define_map(map_name): 
    
    # Initialize project 
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    
    try: 
        mp   = aprx.listMaps(map_name)[0]
    except IndexError: 
        print('Map not found.')
    
    return mp

'''
Clear map 
'''

def clear_map(mp, leave_basemap = True): 
    
    # Clear layers 
    for layer in mp.listLayers(): 
        
        if layer.isBasemapLayer == True and leave_basemap == True: 
            pass 
        else:
            mp.removeLayer(layer)
            
    return mp

'''
Iterate through a focus feature class to select census blocks by distance based on each point in the feature class. 
Returns a list of feature class paths. 
'''

def select_block_groups_by_focus_point(focus_point_name, focus_points, census_path, selection_distance, mp, overwrite, add_to_map):

    # Construct SQL
    sql                          = f"NAME = '{focus_point_name}'"

    # Apply attribute selection 
    focus_selection, count       = arcpy.management.SelectLayerByAttribute(focus_points, 'NEW_SELECTION', sql)

    # Apply block layer selection 
    census_selection, merge, count = arcpy.management.SelectLayerByLocation(census_path, 'WITHIN_A_DISTANCE', focus_selection, 
                                                                            selection_distance, 'NEW_SELECTION')

    print(focus_point_name)
    print(f" >>> {count}")

    # Create feature class name 
    for char in [".", "-", " "]: 
        focus_point_name = focus_point_name.replace(char, "_")

    fc_path = os.path.join(gdb, focus_point_name + "_Block_Groups")

    # Export to feature class 
    if arcpy.Exists(fc_path) and overwrite == False: 
        pass 
    else: 
        try: 
            arcpy.conversion.ExportFeatures(census_selection, fc_path)
        except arcpy.ExecuteError: 
            with arcpy.da.Editor(fc_path) as edit: 
                arcpy.stopEditing(save_changes = True)
            arcpy.conversion.ExportFeatures(census_selection, fc_path)

    # Add to map 
    if add_to_map == True: 
        mp.addDataFromPath(fc_path)
                
    return fc_path

'''
Export shapefiles 
'''

def export_shapefiles(fc_path, shp_folder): 
    
    # Set output path 
    fc_name  = fc_path.split("\\")[-1]
    shp_path = os.path.join(shp_folder, fc_name + ".shp")

    # Export 
    arcpy.conversion.ExportFeatures(fc_path, shp_path) 
    
    # Print 
    print("")
    print(shp_path)
        
    return shp_path